Using TensorFlow backend.


In [ ]:

player = MidiPlayer()
socket = SocketIO('localhost', 9876, LoggingNamespace)



corpus = read_csv('music_data.csv', header=1)
print('corpus length:', len(corpus))

notes_index = corpus.values[:, 0]
# data = data.astype('float32')

notes = sorted(list(set(notes_index)))

print('num_events:', len(notes))
print(notes)
note_index = dict((c, i) for i, c in enumerate(notes))
index_note = dict((i, c) for i, c in enumerate(notes))
print(note_index)
print(index_note)=

# cut the text in semi-redundant sequences of maxlen characters
n_time_steps = 100
semi_redundancy_step = 3
phrases = []
next_note = []
for i in range(0, len(notes_index) - n_time_steps, semi_redundancy_step):
    phrases.append(notes_index[i: i + n_time_steps])
    next_note.append(notes_index[i + n_time_steps])
print('nb sequences:', len(phrases))
for i in range(3):
    print('______________')
    print('x', phrases[i])
    print('y', next_note[i])
#
print('Vectorization...')
x = np.zeros((len(phrases), n_time_steps, len(notes)), dtype=np.bool)
y = np.zeros((len(phrases), len(notes)), dtype=np.bool)
#
for i, phrase in enumerate(phrases):
    for t, note in enumerate(phrase):
        x[i, t, note_index[note]] = 1
    y[i, note_index[next_note[i]]] = 1
for i in range(3):
    print('______________')
    print('x',x[i])
    print('y', y[i])


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(n_time_steps, len(notes)), return_sequences=True))
model.add(LSTM(128, input_shape=(n_time_steps, len(notes))))
model.add(Dense(len(notes)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def midi_to_hertz(midi):
    if midi == 0:
        return 0
    g = 2**(1/12)
    return 440*g**(midi-69)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    if epoch > 17 and epoch % 2 == 0:
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(notes_index) - n_time_steps - 1)
        for diversity in [1, 1.1, 1.2]:
            print('----- diversity:', diversity)

            generated = []
            current_phrase = notes_index[start_index: start_index + n_time_steps]
            generated.extend(current_phrase)
            # print(generated)
            print('----- Generating with seed: ', current_phrase)

            for i in range(700):
                x_pred = np.zeros((1, n_time_steps, len(notes)))
                for t, event in enumerate(current_phrase):
                    x_pred[0, t, note_index[event]] = 1.

                note_pred = model.predict(x_pred, verbose=0)[0]
                note_index = sample(note_pred, diversity)
                next_note = index_note[note_index]
                generated.append(next_note)
                current_phrase = np.append(current_phrase[1:], next_note)

                # sys.stdout.write(str(next_char) + ' ')
                # sys.stdout.flush()

                # print()
            print(generated)
            for value in generated:
                freq = midi_to_hertz(value)
                print(freq, value)
                socket.emit('freq_change', {'freq': freq, 'vol': 100})
                time.sleep(0.03)
                # player.play(value, .03, 70)
            socket.emit('freq_change', {'freq': 0, 'vol': 100})


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=256,
          epochs=300,
          callbacks=[print_callback]
        )
